<a href="https://colab.research.google.com/github/alexisdr/uned-tfg/blob/main/UNED-TFG-2-data-set-tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del Dataset

Durante este proceso se crea un dataset con lo datos procesados y listos para el entrenamiento del modelo.

# Parametros

*  GoldStandardTrainingEAs.txt: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
*  GoldStandardTestEAs.txt: equivalente al anterior pero con los datos del conjunto de pruebas.
*  Training: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
*  Training: equivalente a training pero con el conjunto de pruebas.
* dataset_path: ruta en el que se almacenará el dataset
* labels_file: ruta en la que se almacenará la lista de etiquetas

In [1]:
base_path = '/drive/My Drive/CorpusPFG/'

#Datos de origen
GoldStandardTrainingEA_data_location = base_path + 'Tiny-GoldStandardTrainingEAs.txt'
training_files_location = base_path + 'Tiny-Training/'

#Dataset procesado
dataset_path = base_path + 'Tiny-Dataset'
labels_file = dataset_path + '/labels.txt'

In [2]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.1 MB/s eta 0:00:00


Se realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [3]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


#Creación del dataset




Procedemos a unir en un dataset etiquetado con los EAs la sección de juicio clínico de los informes que componen el acto clínico. Para esto, del fichero GoldStandardTrainingEAs.txt se toman las líneas en la que los actos clínicos tienen un EA asociado (diferentes de NONE). Por cada fichero que compone el acto clínico se extrae el juicio clínico. El resultado serán 2 columnas:

label: Primer EA
text: Frases del juicio clínico, escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".

Carga del archivo de texto GoldStandardTrainingEAs.txt en un data set

In [4]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", 
                "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(
    GoldStandardTrainingEA_data_location, sep=' ', header=None, names=column_names)

Procedemos a unir en un dataset etiquetado con los EAs la sección de juicio clínico de los informes que componen el acto clínico.
Para esto, del fichero GoldStandardTrainingEAs.txt se toman las líneas en la que los actos clínicos tienen un EA asociado (diferentes de NONE). Por cada fichero que compone el acto clínico se extrae el juicio clínico. El resultado serán 2 columnas: 
*   label: Primer EA
*   text: Frases del juicio clínico, escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".



Generación de las etiquetas

In [10]:
df_goldStandardTrainingEAs.TieneEA_EA1.unique()

array(['NONE', 'T38.0X5A', 'K12.31', 'Y95', 'E09.65', 'I97.618', 'G72.0',
       'T88.6XXA', 'G97.82', 'D70.2', 'L27.1', 'T50.2X5A', 'T45.515A',
       'T88.7XXA', 'T39.1X5A', 'T46.0X5A', 'T50.1X5A', 'T36.95XA',
       'T88.8XXS', 'T81.4XXA', 'I97.790', 'T46.2X5D', 'T39.395A',
       'K12.32', 'T50.8X5A', 'M96.840', 'T82.120A', 'K59.03', 'T83.51XA',
       'T50.995A', 'T83.59XA', 'Y84.9', 'P01.1', 'P03.4', 'P36.9',
       'P02.69', 'P02.5', 'P13.4', 'P39.1', 'P03.3', 'P14.1', 'P12.89',
       'T45.1X5A', 'D70.1', 'T50.905A', 'T80.211A', 'K94.21', 'K85.30',
       'T80.219A', 'N99.522', 'T84.038A', 'G62.0', 'T36.8X5A', 'T80.89XA',
       'Y84.2', 'T84.498A', 'Y83.1', 'M96.830', 'T84.223A', 'I97.130',
       'T82.868A', 'J95.811', 'G97.41', 'M96.810', 'T83.89XA', 'I97.190',
       'T82.09XD', 'T84.620A', 'I97.89', 'T82.848A', 'T84.090A',
       'T82.199A', 'T82.110A', 'T85.122A', 'T84.226A', 'L76.22',
       'T84.124A', 'T46.4X5A', 'T82.320A', 'N99.111', 'T82.857A',
       'T84.022A', 'T

In [7]:
import os

os.mkdir(dataset_path)

In [8]:
from datasets import ClassLabel

def generate_label_list():
  df_labels = df_goldStandardTrainingEAs
   
  label_list = list(df_labels.TieneEA_EA1.unique())
  save_label_list(label_list)
  return ClassLabel(num_classes=len(label_list), names=label_list), label_list

def save_label_list(label_list):
  print(labels_file)
  with open(labels_file,'w') as tfile:
	  tfile.write('\n'.join(label_list))

def read_label_list():
  label_list = [line.strip() for line in open(labels_file, 'r')]
  return ClassLabel(num_classes=len(label_list), names=label_list), label_list

class2label, label_list = generate_label_list()

/drive/My Drive/CorpusPFG/Tiny-Dataset/labels.txt


Precesamiento del texto de los informes

In [ ]:
import os

JUICIO_CLINICO = 'Juicio Clínico'
TRATAMIENTO = 'Tratamiento'
FIRMADO_XXX = '_FIRMADO_XXXX_'

TRAINING = 0
VALIDATION = 1

def es_linea_comienzo_seleccion (linea):
  return True #linea.strip() == JUICIO_CLINICO

def es_linea_fin_seleccion (linea):
  return FIRMADO_XXX in linea.strip()

def es_linea_adecuada (linea):
  return linea.strip() != '' #and linea.strip() != JUICIO_CLINICO

def tratar_linea_texto (linea):
  return linea.replace ("- ", "").replace("\n", "").replace('"', '""')

def obtener_texto (acto, label_list, lista_nombres_informes_medicos, files_location):
  #print(f"Acto: {acto} EA: {ea} Archivo: {lista_nombres_informes_medicos}")
  text = ''
  for nombre_informe in lista_nombres_informes_medicos:
    with open (files_location + nombre_informe, 'rt') as informe_medico: 
      seleccionar = False
      for lineaTexto in informe_medico:
        if (es_linea_comienzo_seleccion(lineaTexto)):
          seleccionar = True
        if (es_linea_fin_seleccion(lineaTexto)):
          seleccionar = False
        if (seleccionar and es_linea_adecuada(lineaTexto)):
          text = text + tratar_linea_texto(lineaTexto) + "\n"
      return [acto,
              class2label.str2int(label_list[0].strip()), 
              label_list[0].strip(), 
              label_list, 
              lista_nombres_informes_medicos, text]


def get_label_list(row):
  label_list = list()
  for col in column_names:
    if (col == column_names[0]):
      continue
    if (type(row[col]) == str):
      label_list.append(row[col])
  return label_list

def generate_labeled_array (dataframe, file_list, files_location): 
  dataframe_SinNone = dataframe.reset_index()  
  ds_resultado = []
  for index, row in dataframe_SinNone.iterrows():
    texto_filtro = str(row['Acto']) + '-'
    archivos_acto = filter(lambda x: texto_filtro in x, file_list)       
    ds_resultado.append(obtener_texto(row['Acto'], get_label_list(row), 
                                      list(archivos_acto), files_location))
  return ds_resultado

def save_labeled_dataset (labeled_array, labeled_dataset_name):
  ds = pd.DataFrame(labeled_array, columns=['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'])
  #ds.to_csv(labeled_dataset_name)
  return ds

def generate_labeled_arrays ():
  training_files = [f for f in os.listdir(training_files_location) 
                      if os.path.isfile(os.path.join(training_files_location, f))]
  labeled_array_training = generate_labeled_array (
      df_goldStandardTrainingEAs, training_files, training_files_location)
  ds_labeled_training = save_labeled_dataset (labeled_array_training, eas_dataset_training_file)

  return ds_labeled_training

datasets = generate_labeled_arrays()

Composición del dataset

In [ ]:
from datasets import Dataset, Value, DatasetDict

train_dataset = Dataset.from_pandas(datasets)

def set_dataset_features (dataset):
  new_features = dataset.features.copy()
  new_features["label"] = class2label
  return dataset.cast(new_features)
  
train_dataset = set_dataset_features (train_dataset)
train_val_dataset = train_dataset.train_test_split(test_size=0.1)

dataset_complete = DatasetDict(
    {
        "train": train_val_dataset["train"],
        "validation": train_val_dataset["test"]
    })

dataset_complete

Casting the dataset:   0%|          | 0/2321 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/583 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 2088
    })
    validation: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 233
    })
    test: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 583
    })
})

In [ ]:
dataset_complete["train"][4]

{'acto': 24054248,
 'label': 413,
 'label_str': 'P14.1',
 'labels_str': ['P14.1', 'P01.1', 'P03.2'],
 'informes': ['24054248-161168367.txt'],
 'text': ' _CABECERA_XXXX_  \nSexo :Mujer\n\tINFORME ALTA DE NEONATOLOGÍA\n\tNIF/NIE: \n\tNSS: \n\tCIP: \n\tCIPA: \n\t  _DIRECCION_XXXX_ .P:  _NUMERO_XXXX_   _LOCALIDAD_XXXX_ \n\t_NOMBRE_XXXX_  \n\tTeléfono:  _DATA_XXXX_ \n\tFecha ingreso:  _FECHA_XXXX_   _HORA_XXXX_ \nFecha alta:        _FECHA_XXXX_ \n\t _DOC_XXXX_ \n\tTipo de Ingreso: Recién nacido patológico\n\tMotivo de Ingreso Proceso de Recién Nacido\nMotivo de Alta\n _APELLIDO_XXXX_ \nEdad al ingreso: Recién nacida                   Edad al alta: 6 horas de vida\nMotivo de ingreso: Riesgo de infección\nAntecedentes personales                                     EG: 41+1 semanas\nEmbarazo: Controlado de curso normal. Ecografías prenatales normales, en Ecografía de 2º trimestre donde se visualiza foco hiperecogénico en VI. Serologías negativas, excepto rubeola inmune. Frotis vaginal EGB nega

In [ ]:
dataset_complete.save_to_disk(dataset_path)

Flattening the indices:   0%|          | 0/2088 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2088 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/233 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/233 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/583 [00:00<?, ? examples/s]